In [3]:
! pip install webdriver_manager selenium seleniumbase transformers datasets adapters evaluate transformers torch tensorflow

In [46]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
# write selenium script to submit 16personalities test and get results

from webdriver_manager.chrome import ChromeDriverManager



from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


import time
import os
import random
import json
from tqdm.auto import tqdm

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless modes
chrome_options.add_argument("--disable-animations")  # Disable animations


In [5]:
def _eval(choices, result_name):
    # Set page load strategy to "none"
    driver = webdriver.Chrome(service = ChromeDriverManager().install(), options=chrome_options)
    driver.get("https://www.16personalities.com/free-personality-test")

    # # wait for the page to load
    time.sleep(0.5)

    PAGES = 10

    for page_no in tqdm(range(PAGES)):
        # # click the next button with aria-label="Go to next set of questions"
        # # for all the class="radios" element, print their id and value in label tag
        
        radios = driver.find_elements(By.CLASS_NAME, "radios")
        for r_idx, radio in enumerate(radios):
            # find all input tags and store it as (id, value) pair
            inputs = radio.find_elements(By.TAG_NAME, "input")
            opt = []
            # for input in inputs:
            #     opt.append((input.get_attribute("id"), input.get_attribute("value")))
            # print(opt)
            labels = radio.find_elements(By.TAG_NAME, "label")

            # click on label with value="1"
            # VAL = -2
            # VAL = choices[page_no][r_idx]
            VAL = choices[page_no*6 + r_idx]
            label = labels[VAL + 3]
            driver.execute_script("arguments[0].click();", label)            


        time.sleep(0.1)
        if page_no != PAGES - 1:
            next_button = driver.find_element(By.XPATH, "//button[@aria-label='Go to next set of questions']")
            driver.execute_script("arguments[0].click();", next_button)
        else:
            # click on the button type="submit"
            submit_button = driver.find_element(By.XPATH, "//button[@type='submit']")
            driver.execute_script("arguments[0].click();", submit_button)

    # wait for the page to load
    time.sleep(0.2)

    # get the result, save the html page


    RESULT_PAGES = 6
    SAVE_DIR = "results"
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    result = {}

    for page_no in range(RESULT_PAGES):
        time.sleep(0.2)

        # result = driver.page_source
        # with open(f"{SAVE_DIR}/result_{page_no}.html", "w") as f:
        #     f.write(result)

        if page_no == 0:
            # title is "results__type__name" is present in class attribute
            title = driver.find_element(By.CLASS_NAME, "results__type__name").text

            # mbti type results__type__code is present in class attribute
            mbti = driver.find_element(By.CLASS_NAME, "results__type__code").text
            mbti = mbti.split("\n")[1]

            result["title"] = title
            result["mbti_subtype"] = mbti 
            if '-' in mbti:
                result["mbti"] = mbti.split("-")[0]
            else:
                result["mbti"] = mbti
            # print(result["title"], result["mbti"])

        else:
            # card_name is h3 tag with "card__title" inside class attribute
            card_name = driver.find_element(By.CLASS_NAME, "card__title").text

            # parity percentage is inside class="percentage right active"
            pairty_percentage = driver.find_element(By.CLASS_NAME, "percentage").text


            result[card_name] = {
                "pairty_percentage": pairty_percentage,
            }
            # print(card_name, pairty_percentage)


        # click button with class="sp-action sp-button button--action button--blue button--md button--pill button--auto"
        next_button = driver.find_element(By.XPATH, "//button[@class='sp-action sp-button button--action button--blue button--md button--pill button--auto']")
        driver.execute_script("arguments[0].click();", next_button)

    # save the final result in json format
    import json
    with open(f"{SAVE_DIR}/result_{result_name}.json", "w") as f:
        json.dump(result, f)

    driver.quit()
    return result

In [47]:
from copy import deepcopy

def eval_opt(choices, result_name):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.16personalities.com/free-personality-test")

    PAGES = 10
    for page_no in tqdm(range(PAGES)):
        radios = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "radios")))
        for r_idx, radio in enumerate(radios):
            labels = radio.find_elements(By.TAG_NAME, "label")
            VAL = choices[page_no * 6 + r_idx]
            try:
                label = labels[VAL + 3]
            except Exception as e:
                print(VAL)
                raise e
            driver.execute_script("arguments[0].click();", label)

        if page_no != PAGES - 1:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Go to next set of questions']"))
            )
            driver.execute_script("arguments[0].click();", next_button)
        else:
            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
            )
            driver.execute_script("arguments[0].click();", submit_button)

    RESULT_PAGES = 6
    SAVE_DIR = "results"
    os.makedirs(SAVE_DIR, exist_ok=True)
    result = {}

    for page_no in range(RESULT_PAGES):
        if page_no == 0:
            title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "results__type__name"))).text
            mbti = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "results__type__code"))).text
            mbti = mbti.split("\n")[1]
            result["title"] = title
            result["mbti_subtype"] = mbti
            result["mbti"] = mbti.split("-")[0] if '-' in mbti else mbti
        else:
            card_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "card__title"))).text
            parity_percentage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "percentage"))).text
            result[card_name] = {"parity_percentage": parity_percentage}

        if page_no != RESULT_PAGES - 1:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@class='sp-action sp-button button--action button--blue button--md button--pill button--auto']"))
            )
            driver.execute_script("arguments[0].click();", next_button)

    result_simple = {}
    result_simple["mbti_subtype"] = result["mbti_subtype"]
    result_simple["Extroversion"] = result["Energy"]["parity_percentage"]
    result_simple["Intuitive"] = result["Mind"]["parity_percentage"]
    result_simple["Thinker"] = result["Nature"]["parity_percentage"]
    result_simple["Judger"] = result["Tactics"]["parity_percentage"]
    result_simple["Assertive"] = result["Identity"]["parity_percentage"]
    
    with open(f"{SAVE_DIR}/result_{result_name}.json", "w") as f:
        json.dump(result, f)
        
    with open(f"{SAVE_DIR}/result_{result_name}_simple.json", "w") as f:
        json.dump(result_simple, f)

    driver.quit()
    #return result
    return result_simple

In [108]:
# generate random choices
# choices = [[random.randint(-3, 3) for _ in range(6)] for _ in range(10)]
# choices = [random.randint(-3, 3) for _ in range(60)]
# print(choices)



In [83]:
# eval(choices, "random")

In [48]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
import json
import datasets
import adapters
import transformers

In [9]:
from transformers import BertConfig
from adapters import AutoAdapterModel

config = BertConfig.from_pretrained(
    "bert-base-uncased",
    num_labels=7,
)
model = AutoAdapterModel.from_pretrained(
    "bert-base-uncased",
    config=config,
)

/Users/divyajyotipanda/miniconda3/envs/nlpproj/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
ADAPTER_SAVE_PATH = 'new_ia3'

In [37]:
from copy import deepcopy

answer_mapping = {
    "Strongly Disagree": 0,
    "Disagree": 1,
    "Slightly Disagree": 2,
    "Neutral": 3,
    "Slightly Agree": 4,
    "Agree": 5,
    "Strongly Agree": 6
}

label2id = deepcopy(answer_mapping)
id2label = {idx: lbl for lbl, idx in label2id.items()}

In [12]:
from transformers import AutoModelForCausalLM

#model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

TASK_NAME = 'INFP1_1_1_7'

adapters.init(model)

#model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/best_model_dir")
model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
model.set_active_adapters(f"{TASK_NAME}")

# model.add_classification_head(
#     TASK_NAME,
#     num_labels=7,
#     id2label=id2label
# )

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [42]:
questions = [
    "You regularly make new friends.",
    "Complex and novel ideas excite you more than simple and straightforward ones.",
    "You usually feel more persuaded by what resonates emotionally with you than by factual arguments.",
    "Your living and working spaces are clean and organized.",
    "You usually stay calm, even under a lot of pressure.",
    "You find the idea of networking or promoting yourself to strangers very daunting.",
    "You prioritize and plan tasks effectively, often completing them well before the deadline.",
    "People’s stories and emotions speak louder to you than numbers or data.",
    "You like to use organizing tools like schedules and lists.",
    "Even a small mistake can cause you to doubt your overall abilities and knowledge.",
    "You feel comfortable just walking up to someone you find interesting and striking up a conversation.",
    "You are not too interested in discussions about various interpretations of creative works.",
    "You prioritize facts over people’s feelings when determining a course of action.",
    "You often allow the day to unfold without any schedule at all.",
    "You rarely worry about whether you make a good impression on people you meet.",
    "You enjoy participating in team-based activities.",
    "You enjoy experimenting with new and untested approaches.",
    "You prioritize being sensitive over being completely honest.",
    "You actively seek out new experiences and knowledge areas to explore.",
    "You are prone to worrying that things will take a turn for the worse.",
    "You enjoy solitary hobbies or activities more than group ones.",
    "You cannot imagine yourself writing fictional stories for a living.",
    "You favor efficiency in decisions, even if it means disregarding some emotional aspects.",
    "You prefer to do your chores before allowing yourself to relax.",
    "In disagreements, you prioritize proving your point over preserving the feelings of others.",
    "You usually wait for others to introduce themselves first at social gatherings.",
    "Your mood can change very quickly.",
    "You are not easily swayed by emotional arguments.",
    "You often end up doing things at the last possible moment.",
    "You enjoy debating ethical dilemmas.",
    "You usually prefer to be around others rather than on your own.",
    "You become bored or lose interest when the discussion gets highly theoretical.",
    "When facts and feelings conflict, you usually find yourself following your heart.",
    "You find it challenging to maintain a consistent work or study schedule.",
    "You rarely second-guess the choices that you have made.",
    "Your friends would describe you as lively and outgoing.",
    "You are drawn to various forms of creative expression, such as writing.",
    "You usually base your choices on objective facts rather than emotional impressions.",
    "You like to have a to-do list for each day.",
    "You rarely feel insecure.",
    "You avoid making phone calls.",
    "You enjoy exploring unfamiliar ideas and viewpoints.",
    "You can easily connect with people you have just met.",
    "If your plans are interrupted, your top priority is to get back on track as soon as possible.",
    "You are still bothered by mistakes that you made a long time ago.",
    "You are not too interested in discussing theories on what the world could look like in the future.",
    "Your emotions control you more than you control them.",
    "When making decisions, you focus more on how the affected people might feel than on what is most logical or efficient.",
    "Your personal work style is closer to spontaneous bursts of energy than organized and consistent efforts.",
    "When someone thinks highly of you, you wonder how long it will take them to feel disappointed in you.",
    "You would love a job that requires you to work alone most of the time.",
    "You believe that pondering abstract philosophical questions is a waste of time.",
    "You feel more drawn to busy, bustling atmospheres than to quiet, intimate places.",
    "If a decision feels right to you, you often act on it without needing further proof.",
    "You often feel overwhelmed.",
    "You complete things methodically without skipping over any steps.",
    "You prefer tasks that require you to come up with creative solutions rather than follow concrete steps.",
    "You are more likely to rely on emotional intuition than logical reasoning when making a choice.",
    "You struggle with deadlines.",
    "You feel confident that things will work out for you."
]
questions = [question.lower() for question in questions]

In [15]:
import torch

# i = 0
answers = []
for input_text in questions:
    # if i % 6 == 0:
    #     print("Page", i // 6 + 1)

    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=80, padding="max_length")
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Make prediction
    with torch.no_grad():  # Disables gradient calculation to save memory and computations
        outputs = model(**inputs)

    # Extract logits (model outputs prior to activation function)
    answers.append(3 - torch.argmax(outputs.logits).item())
    # print(answer)

    # i += 1
    # if i % 6 == 0:
    #     print()
    #     print()

In [94]:
answers

[1,
 0,
 -1,
 3,
 2,
 2,
 3,
 -1,
 1,
 0,
 2,
 2,
 -3,
 1,
 2,
 2,
 0,
 2,
 -3,
 -3,
 2,
 -1,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 -3,
 2,
 1,
 2,
 3,
 -3,
 1,
 -1,
 -3,
 1,
 2,
 2,
 -3,
 2,
 3,
 0,
 -3,
 -3,
 3,
 1,
 2,
 -2,
 2,
 1,
 3,
 -3,
 1,
 -2,
 -3,
 1,
 3]

In [16]:
eval_opt(answers, TASK_NAME)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

{'mbti_subtype': 'INTP-T',
 'Extroversion': '41%',
 'Intuitive': '61%',
 'Thinker': '65%',
 'Judger': '39%',
 'Assertive': '32%'}

In [17]:
from joblib import delayed, parallel
def evaluation_wrapper(_lambda):
    traits = [[("EXTROVERSION", "E"), ("INTROVERSION", "I")], [("SENSOR", "S"), ("INTUITIVE", "N")], [("THINKER", "T"), ("FEELER", "F")], [("JUDGING", "J"), ("PERCEIVER", "P")]]
    personality_results = {}
    for tr1 in traits[0]:
        for tr2 in traits[1]:
            for tr3 in traits[2]:
                for tr4 in traits[3]:
                    task_name = [x[0] for x in [tr1, tr2, tr3, tr4]]
                    personality = "".join([x[1] for x in [tr1, tr2, tr3, tr4]]) + "_".join([str(x) for x in _lambda])
                    config = BertConfig.from_pretrained(
                        "bert-base-uncased",
                        num_labels=7,
                    )
                    model = AutoAdapterModel.from_pretrained(
                        "bert-base-uncased",
                        config=config,
                    )
                    
                    TASK_NAME = personality

                    adapters.init(model)

                    print(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
                    model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
                    model.set_active_adapters(f"{TASK_NAME}")
                    
                    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
                    
                    answers = []
                    for input_text in questions:
                        inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=80, padding="max_length")
                        
                        inputs = {k: v.to(model.device) for k, v in inputs.items()}

                        with torch.no_grad():  # Disables gradient calculation to save memory and computations
                            outputs = model(**inputs)

                        answers.append(3 - torch.argmax(outputs.logits).item())
                    try:
                        eval_result = eval_opt(answers, TASK_NAME)
                    except:
                        print(f"[ERROR]{personality} to do manually again.")
                        eval_result = {"error": "to do again"}
                    
                    personality_results[personality[:4]] = eval_result
                    
    return personality_results

In [18]:
evaluation_wrapper([1, 1, 1, 7])

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


new_ia3/ESTJ1_1_1_7/ESTJ1_1_1_7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


new_ia3/INFP1_1_1_7/INFP1_1_1_7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

{'ESTJ': {'mbti_subtype': 'ISFJ-A',
  'Extroversion': '48%',
  'Intuitive': '37%',
  'Thinker': '46%',
  'Judger': '64%',
  'Assertive': '72%'},
 'ESTP': {'mbti_subtype': 'INTP-T',
  'Extroversion': '41%',
  'Intuitive': '61%',
  'Thinker': '65%',
  'Judger': '39%',
  'Assertive': '32%'},
 'ESFJ': {'mbti_subtype': 'ISFJ-A',
  'Extroversion': '48%',
  'Intuitive': '37%',
  'Thinker': '46%',
  'Judger': '64%',
  'Assertive': '72%'},
 'ESFP': {'error': 'to do again'},
 'ENTJ': {'mbti_subtype': 'ISFJ-A',
  'Extroversion': '48%',
  'Intuitive': '37%',
  'Thinker': '46%',
  'Judger': '64%',
  'Assertive': '72%'},
 'ENTP': {'mbti_subtype': 'INTP-T',
  'Extroversion': '41%',
  'Intuitive': '61%',
  'Thinker': '65%',
  'Judger': '39%',
  'Assertive': '32%'},
 'ENFJ': {'mbti_subtype': 'ISFJ-A',
  'Extroversion': '48%',
  'Intuitive': '37%',
  'Thinker': '46%',
  'Judger': '64%',
  'Assertive': '72%'},
 'ENFP': {'error': 'to do again'},
 'ISTJ': {'mbti_subtype': 'ISFJ-A',
  'Extroversion': '48%',

In [19]:
from joblib import Parallel, delayed
import torch
from transformers import BertConfig, AutoTokenizer, AutoModelForSequenceClassification as AutoAdapterModel

# Assuming `adapters`, `eval_opt`, `questions`, `ADAPTER_SAVE_PATH`, and any other necessary imports or definitions are provided elsewhere in your code.

# Evaluation logic wrapped in a function for parallel execution
def evaluate_personality(_lambda, tr1, tr2, tr3, tr4):
    try:
        personality = "".join([x[1] for x in [tr1, tr2, tr3, tr4]]) + "_".join([str(x) for x in _lambda])
        config = BertConfig.from_pretrained(
            "bert-base-uncased",
            num_labels=7,
        )
        model = AutoAdapterModel.from_pretrained(
            "bert-base-uncased",
            config=config,
        )
        
        TASK_NAME = personality
        adapters.init(model)
        print(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
        model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
        model.set_active_adapters(TASK_NAME)
        
        tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        
        answers = []
        for input_text in questions:
            inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=80, padding="max_length")
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            with torch.no_grad():  # Disables gradient calculation to save memory and computations
                outputs = model(**inputs)
            answers.append(3 - torch.argmax(outputs.logits).item())
        try:
            eval_result = eval_opt(answers, TASK_NAME)
        except Exception as e:
            print(f"[ERROR]{personality} to do manually again. Error: {e}")
            eval_result = {"error": "to do again"}
    except Exception as e:
        print(f"[ERROR]{personality} to do manually again. Error: {e}")
        eval_result = {"error": e}
        
        
    print(f"{personality[:4]}: {eval_result}")

    return personality[:4], eval_result

# Main function to execute parallel evaluation
def parallel_evaluation(_lambda):
    traits = [[("EXTROVERSION", "E"), ("INTROVERSION", "I")], [("SENSOR", "S"), ("INTUITIVE", "N")], [("THINKER", "T"), ("FEELER", "F")], [("JUDGING", "J"), ("PERCEIVER", "P")]]
    # tasks = [delayed(evaluate_personality)(_lambda, tr1, tr2, tr3, tr4)
    #          for tr1 in traits[0]
    #          for tr2 in traits[1]
    #          for tr3 in traits[2]
    #          for tr4 in traits[3]]
    # results = Parallel(n_jobs=-1)(tasks)
    # personality_results = {personality: result for personality, result in results}
    personality_results = {}
    for tr1 in traits[0]:
        for tr2 in traits[1]:
            for tr3 in traits[2]:
                for tr4 in traits[3]:
                    personality, result = evaluate_personality(_lambda, tr1, tr2, tr3, tr4)
                    personality_results[personality] = result
    return personality_results

# # Example usage
# _lambda_example = [3,2,2,3]
# results = parallel_evaluation(_lambda_example)
# print(results)

In [20]:
dict_eval = {}
for l1 in range(1, 10):
    for l2 in range(1, 10 - l1):
        for l3 in range(1, 10 - l1 - l2):
            #print([l1 / 20, l2 / 20, l3 / 20, (20 - l1 - l2 - l3) / 20])
            print([l1, l2, l3, 10 - l1 - l2 - l3])
            dict_eval[(l1, l2, l3, 10 - l1 - l2 - l3)] = parallel_evaluation([l1, l2, l3, 10 - l1 - l2 - l3])
            print()
            print()
            print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[1, 1, 1, 7]


Model class 'BertAdapterModel' of found prediction head does not match current model class.


new_ia3/ESTJ1_1_1_7/ESTJ1_1_1_7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Model class 'BertAdapterModel' of found prediction head does not match current model class.


new_ia3/INFP7_1_1_1/INFP7_1_1_1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

INFP: {'mbti_subtype': 'ISTP-T', 'Extroversion': '45%', 'Intuitive': '34%', 'Thinker': '55%', 'Judger': '46%', 'Assertive': '42%'}





In [43]:
def fix_issues(d):
    for _lambda in d:
        for personality in d[_lambda]:
            if len(d[_lambda][personality]) < 5 or type(d[_lambda][personality]) != type({}):
                d[_lambda][personality] = {}
                pers = [(_, personality[i]) for i in range(4)]
                #d[_lambda][personality] = evaluate_personality(_lambda, pers)
                d[_lambda][personality] = evaluate_personality(_lambda, *pers)[1]
    return d
                
dict_eval = fix_issues(dict_eval)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Model class 'BertAdapterModel' of found prediction head does not match current model class.


new_ia3/ESFJ1_1_2_6/ESFJ1_1_2_6


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

ESFJ: {'mbti_subtype': 'ENFJ-T', 'Extroversion': '56%', 'Intuitive': '51%', 'Thinker': '44%', 'Judger': '58%', 'Assertive': '42%'}


...

Model class 'BertAdapterModel' of found prediction head does not match current model class.


new_ia3/ENTJ4_4_1_1/ENTJ4_4_1_1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

ENTJ: {'mbti_subtype': 'ISTP-T', 'Extroversion': '47%', 'Intuitive': '42%', 'Thinker': '53%', 'Judger': '49%', 'Assertive': '49%'}


In [44]:
def count_issues(d):
    num_issues = 0
    for _lambda in d:
        for personality in d[_lambda]:
            if len(d[_lambda][personality]) < 5 or type(d[_lambda][personality]) != type({}):
                # print(d[_lambda])
                num_issues += 1
    return num_issues
                
count_issues(dict_eval)

0

In [45]:
def flip_params(d):
    new_dict = {}
    for _lambda in d:
        for personality in d[_lambda]:
            if personality not in new_dict:
                new_dict[personality] = {}
            new_dict[personality][_lambda] = d[_lambda][personality]
            
    return new_dict

In [46]:
flipped_dict_eval = flip_params(dict_eval)

In [49]:
flipped_dict_eval['ESTJ'][(1, 1, 1, 7)].keys()

dict_keys(['mbti_subtype', 'Extroversion', 'Intuitive', 'Thinker', 'Judger', 'Assertive'])

In [50]:
def expand_dict(d):
    return {
        personality: {
            _lambda: {
                'E': int(d[personality][_lambda]['Extroversion'][:-1]), 
                'I': 100 - int(d[personality][_lambda]['Extroversion'][:-1]),
                'N': int(d[personality][_lambda]['Intuitive'][:-1]),
                'S': 100 - int(d[personality][_lambda]['Intuitive'][:-1]),
                'T': int(d[personality][_lambda]['Thinker'][:-1]),
                'F': 100 - int(d[personality][_lambda]['Thinker'][:-1]),
                'J': int(d[personality][_lambda]['Judger'][:-1]),
                'P': 100 - int(d[personality][_lambda]['Judger'][:-1])
            } for _lambda in d[personality]
        } for personality in d
    }
    
expanded_dict_eval = expand_dict(flipped_dict_eval)

In [53]:
def filter_dict(d):
    new_dict = {}
    for personality in d:
        new_dict[personality] = {}
        for _lambda in d[personality]:
            cond = True
            for tr in personality:
                if d[personality][_lambda][tr] < 50:
                    cond = False
                    break
                
            if cond:
                new_dict[personality][_lambda] = d[personality][_lambda]
    return new_dict

filter_dict_eval = filter_dict(expanded_dict_eval)

In [82]:
misclassified_personalities = [personality for personality in filter_dict_eval if len(filter_dict_eval[personality]) == 0]
misclassified_personalities

['ENTP', 'ISFP', 'INFP']

In [87]:
max_param = {}
for personality in filter_dict_eval:
    if len(filter_dict_eval[personality]) != 0:
        key_max = max(filter_dict_eval[personality].keys(), key = lambda x: sum(filter_dict_eval[personality][x][tr] for tr in personality))
        max_param[personality] = key_max
    else:
        max_param[personality] = None

max_param

{'ESTJ': (1, 4, 2, 3),
 'ESTP': (4, 1, 3, 2),
 'ESFJ': (1, 5, 2, 2),
 'ESFP': (1, 3, 3, 3),
 'ENTJ': (5, 1, 2, 2),
 'ENTP': None,
 'ENFJ': (1, 1, 3, 5),
 'ENFP': (3, 2, 1, 4),
 'ISTJ': (1, 7, 1, 1),
 'ISTP': (2, 3, 2, 3),
 'ISFJ': (6, 2, 1, 1),
 'ISFP': None,
 'INTJ': (4, 4, 1, 1),
 'INTP': (1, 7, 1, 1),
 'INFJ': (1, 4, 3, 2),
 'INFP': None}

In [72]:
key = 'ESTJ'
flipped_dict_eval[key][max_param[key]]

{'mbti_subtype': 'ESTJ-T',
 'Extroversion': '63%',
 'Intuitive': '36%',
 'Thinker': '57%',
 'Judger': '67%',
 'Assertive': '49%'}

In [95]:
def filter_dict_three_match(d):
    new_dict = {}
    for personality in d:
        new_dict[personality] = {}
        for _lambda in d[personality]:
            cond = 2
            score = 0
            for tr in personality:
                if d[personality][_lambda][tr] < 50:
                    cond -= 1
                    if cond == 0:
                        break
                else:
                    score += d[personality][_lambda][tr]
            if cond > 0:
                new_dict[personality][_lambda] = d[personality][_lambda]
                new_dict[personality][_lambda]["score"] = score
    return new_dict

expanded_dict_eval_three_match = {x: expanded_dict_eval[x] for x in expanded_dict_eval if max_param[x] is None}
filter_dict_eval_three_match = filter_dict_three_match(expanded_dict_eval_three_match)
filter_dict_eval_three_match

{'ENTP': {(1, 2, 4, 3): {'E': 60,
   'I': 40,
   'N': 34,
   'S': 66,
   'T': 57,
   'F': 43,
   'J': 42,
   'P': 58,
   'score': 175},
  (1, 2, 5, 2): {'E': 52,
   'I': 48,
   'N': 55,
   'S': 45,
   'T': 53,
   'F': 47,
   'J': 53,
   'P': 47,
   'score': 160},
  (1, 5, 2, 2): {'E': 73,
   'I': 27,
   'N': 51,
   'S': 49,
   'T': 53,
   'F': 47,
   'J': 67,
   'P': 33,
   'score': 177},
  (2, 3, 3, 2): {'E': 66,
   'I': 34,
   'N': 66,
   'S': 34,
   'T': 58,
   'F': 42,
   'J': 69,
   'P': 31,
   'score': 190},
  (2, 4, 2, 2): {'E': 48,
   'I': 52,
   'N': 51,
   'S': 49,
   'T': 57,
   'F': 43,
   'J': 44,
   'P': 56,
   'score': 164},
  (2, 4, 3, 1): {'E': 48,
   'I': 52,
   'N': 54,
   'S': 46,
   'T': 52,
   'F': 48,
   'J': 31,
   'P': 69,
   'score': 175},
  (3, 1, 1, 5): {'E': 47,
   'I': 53,
   'N': 61,
   'S': 39,
   'T': 52,
   'F': 48,
   'J': 47,
   'P': 53,
   'score': 166},
  (3, 1, 5, 1): {'E': 45,
   'I': 55,
   'N': 51,
   'S': 49,
   'T': 55,
   'F': 45,
   'J': 42

In [96]:
misclassified_personalities_three_match = [personality for personality in filter_dict_eval_three_match if len(filter_dict_eval_three_match[personality]) == 0]
misclassified_personalities_three_match

[]

In [107]:
max_param_three_match = {}
for personality in filter_dict_eval_three_match:
    if len(filter_dict_eval_three_match[personality]) != 0:
        key_max = max(filter_dict_eval_three_match[personality].keys(), key = lambda x: filter_dict_eval_three_match[personality][x]['score'])
        max_param_three_match[personality] = key_max
    else:
        max_param_three_match[personality] = None

max_param_three_match

{'ENTP': (2, 3, 3, 2), 'ISFP': (4, 1, 3, 2), 'INFP': (1, 2, 3, 4)}

In [108]:
key = 'ENTP'
flipped_dict_eval[key][max_param_three_match[key]]

{'mbti_subtype': 'ENTJ-T',
 'Extroversion': '66%',
 'Intuitive': '66%',
 'Thinker': '58%',
 'Judger': '69%',
 'Assertive': '49%'}

In [101]:
best_models = {}
for key in max_param:
    if max_param[key] is not None:
        best_models[key] = max_param[key]
    else:
        best_models[key] = max_param_three_match[key]
best_models

{'ESTJ': (1, 4, 2, 3),
 'ESTP': (4, 1, 3, 2),
 'ESFJ': (1, 5, 2, 2),
 'ESFP': (1, 3, 3, 3),
 'ENTJ': (5, 1, 2, 2),
 'ENTP': (2, 3, 3, 2),
 'ENFJ': (1, 1, 3, 5),
 'ENFP': (3, 2, 1, 4),
 'ISTJ': (1, 7, 1, 1),
 'ISTP': (2, 3, 2, 3),
 'ISFJ': (6, 2, 1, 1),
 'ISFP': (4, 1, 3, 2),
 'INTJ': (4, 4, 1, 1),
 'INTP': (1, 7, 1, 1),
 'INFJ': (1, 4, 3, 2),
 'INFP': (1, 2, 3, 4)}

In [112]:
dict_names = {key: key + "_".join([str(x) for x in best_models[key]]) for key in best_models}
dict_names
with open('task_names.json', 'w') as f:
    json.dump(dict_names, f)

In [25]:
import json
import shutil

with open('task_names.json', 'r') as f:
    dict_names = json.load(f)

for folder in dict_names.values():
    path = folder
    shutil.copytree('new_ia3/' + folder, 'merged_models_ia3/' + folder)

In [26]:
for folder in dict_names.values():
    path = "results/result_" + folder + "_simple.json"
    shutil.copy(path, 'merged_models_ia3/' + path.split('/')[-1])

In [65]:
from transformers import AutoTokenizer
from transformers import BertConfig
from adapters import AutoAdapterModel
import json
import adapters

ADAPTER_SAVE_PATH = 'new_ia3'

def multi_eval_opt(TASK_NAME, N):
    result_dict = {x: [] for x in ['Extroversion', 'Intuitive', 'Thinker', 'Judger', 'Assertive']}
    count = 0
    for _ in range(N * 10):
        try:
            config = BertConfig.from_pretrained(
                "bert-base-uncased",
                num_labels=7,
            )
            model = AutoAdapterModel.from_pretrained(
                "bert-base-uncased",
                config=config,
            )
            
            # adapters.init(model)

            # model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/{TASK_NAME}")
            # #model.load_adapter(f"{ADAPTER_SAVE_PATH}/{TASK_NAME}/best_model_dir")
            # model.set_active_adapters(f"{TASK_NAME}")
            
            model.add_classification_head(
                TASK_NAME,
                num_labels=7,
                id2label=id2label
            )

            tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

            answers = []
            for input_text in questions:
                inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=80, padding="max_length")
                inputs = {k: v.to(model.device) for k, v in inputs.items()}

                with torch.no_grad():  # Disables gradient calculation to save memory and computations
                    outputs = model(**inputs)

                answers.append(3 - torch.argmax(outputs.logits).item())

            ans = eval_opt(answers, TASK_NAME)
            for key in result_dict:
                result_dict[key].append(int(ans[key][:-1]))
            count += 1
        except Exception as e:
            print(e)
        if count == N:
            with open('multi_results/result_' + TASK_NAME + '.json', 'w') as f:
                json.dump(result_dict, f)
            return result_dict
    raise RuntimeError("Reached max number of retries")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [66]:
import torch 
result = multi_eval_opt('baseline', 10)
result

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

1
list index out of range


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 

In [70]:
from statistics import mean, stdev

lst = [47, 44, 51, 63, 31, 49, 53, 51, 47, 54]

print(mean(lst), stdev(lst))

49 8.178562764256865


In [ ]:
45+/-5, 42+/-8, 54+/-7 49 +/-8